# Requerimientos

## Importar librerías 

In [1]:
import pandas as pd
import mysql.connector
import to_db
import requests

## Definir DataFrames

In [2]:
events = pd.read_csv("ufc_events.csv")
fighters = pd.read_csv("ufc_fighters.csv")
fights = pd.read_csv("ufc_fights.csv")

# Limpieza y manipulación de datos

## Quitar datos duplicados o reduntantes

### Events

In [181]:
events = events.drop_duplicates().reset_index(drop=True)

### Fights

In [182]:
fights = fights.drop_duplicates().reset_index(drop=True)

## Arreglar errores estructurales

### Events

#### Cambiar tipos de datos

##### Convertir a Datetime

In [183]:
events['date'] = pd.to_datetime(events['date'])

#### Eventos faltantes

In [184]:
events_clean = events['name'].str.strip().str.replace(r'\s+', ' ', regex=True)
fights_clean = fights['event'].dropna().str.strip().str.replace(r'\s+', ' ', regex=True)

missing_event = set(fights_clean) - set(events_clean)
missing_event

{'UFC Fight Night: Figueiredo vs. Benavidez 2',
 'UFC Fight Night: Kattar vs. Ige',
 'UFC Fight Night: Ngannou vs. Dos Santos'}

In [185]:
missing_events = {
    'events': [
        {'name': 'UFC Fight Night: Figueiredo vs. Benavidez 2',
         'date': '2020-07-18T21:00Z',
         'country': 'United Arab Emirates',
         'city': 'Abu Dhabi',
         'venue': 'Flash Forum'},
        {'name': 'UFC Fight Night: Kattar vs. Ige',
         'date': '2020-07-15T23:00Z',
         'country': 'United Arab Emirates',
         'city': 'Abu Dhabi',
         'venue': 'Flash Forum'},
        {'name': 'UFC Fight Night: Ngannou vs. Dos Santos',
         'date': '2019-06-29T22:00Z',
         'country': 'United States',
         'city': 'Minneapolis',
         'venue': 'Target Center'}
    ]
}

In [186]:
events = pd.concat([events, pd.DataFrame(missing_events['events'])], ignore_index=True)

In [187]:
len(fights.dropna(subset=['event']).groupby('event')) - len(events)

0

### Fighters

#### Cambiar tipos de datos

##### Convertir a INT

In [188]:
int_fighters_cols = ['age', 'ko_tko_wins', 'decision_wins', 'submission_wins']
fighters[int_fighters_cols] = fighters[int_fighters_cols].astype("Int64")

##### Convertir a Datetime

In [189]:
fighters['ufc_debut_date'] = fighters['ufc_debut_date'].apply(pd.to_datetime, errors='coerce')

#### Separar "professional_record" en columnas

In [190]:
# Obtener el índice de la columnas
pos = fighters.columns.get_loc('professional_record')

# Crear DataFrame con columnas respectivas para el record
record = fighters['professional_record'].str.extract(r'(\d+)-(\d+)-(\d+)')
record.columns = ['wins', 'losses', 'draws']
record = record.astype('Int64')

# Insertar nuevas columnas en el DataFrame sucecivamente a la columna base
for i, col in enumerate(record):
    fighters.insert(pos + i, col, record[col])

# Eliminar columnas base del DataFrame 
fighters = fighters.drop(columns=['professional_record'])

### Fights

#### Cambiar tipos de datos

##### Convertir a INT

In [191]:
int_fights_cols = ['possible_rounds', 'fight_order', 'round', 'total_strikes_attempted_red', 
                   'total_strikes_landed_red', 'significant_strikes_attempted_red', 
                   'significant_strikes_landed_red', 'knockdowns_red', 
                   'head_significant_strikes_red', 'body_significant_strikes_red', 
                   'leg_significant_strikes_red', 'distance_significant_strikes_red', 
                   'clinch_significant_strikes_red', 'ground_significant_strikes_red', 
                   'takedowns_attempted_red', 'takedowns_landed_red', 'submission_attempts_red', 
                   'reversals_red', 'total_strikes_attempted_blue', 'total_strikes_landed_blue',
                   'significant_strikes_attempted_blue', 'significant_strikes_landed_blue',
                   'knockdowns_blue', 'head_significant_strikes_blue',
                   'body_significant_strikes_blue', 'leg_significant_strikes_blue',
                   'distance_significant_strikes_blue', 'clinch_significant_strikes_blue',
                   'ground_significant_strikes_blue', 'takedowns_attempted_blue',
                   'takedowns_landed_blue', 'submission_attempts_blue', 'reversals_blue']

fights[int_fights_cols] = fights[int_fights_cols].astype('Int64')

##### Convertir a Timedelta

In [192]:
time_fights_cols = ['time', 'clinch_control_time_red', 'ground_control_time_red',
                    'clinch_control_time_blue', 'ground_control_time_blue']

fights[time_fights_cols] = fights[time_fights_cols].apply(lambda col: pd.to_timedelta("00:" + col))

##### Convertir a Bool

In [193]:
bool_fights_cols = ['fight_of_the_night', 'ko_of_the_night_red', 'submission_of_the_night_red', 
                    'performance_of_the_night_red', 'ko_of_the_night_blue', 
                    'submission_of_the_night_blue', 'performance_of_the_night_blue']

fights[bool_fights_cols] = fights[bool_fights_cols].astype("bool")

## Manejar valores NaN

### Events

In [194]:
events.isna().sum()

name       0
date       0
country    0
city       0
venue      0
dtype: int64

### Fighters

In [195]:
fighters.isna().sum()

name                  0
nickname           1267
place_of_birth      187
age                 260
height              315
reach              1152
leg_reach          1434
fighting_style     2052
ufc_debut_date        0
wins                 40
losses               40
draws                40
ko_tko_wins          49
decision_wins        49
submission_wins      49
is_active             0
dtype: int64

#### wins/losses/draws

In [196]:
# Obtener los nombres de los peleadores cuyo record sea NaN
wins_nan = fighters[fighters['wins'].isna()]['name']
losses_nan = fighters[fighters['losses'].isna()]['name']
draws_nan = fighters[fighters['draws'].isna()]['name']

# Verificar que los valores faltantes correspondan a los mismos peleadores en todos los casos
set(wins_nan) == set(losses_nan) == set(draws_nan)

True

In [197]:
fighters[fighters['wins'].isna()].sort_values(by='ufc_debut_date')

,name,nickname,place_of_birth,age,height,reach,leg_reach,fighting_style,ufc_debut_date,wins,losses,draws,ko_tko_wins,decision_wins,submission_wins,is_active
1307,Asikeerbai Jinensibieke,NaN,"Xinjiang, China",34,NaN,NaN,NaN,NaN,2022-06-10,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1705,Dallas Marron,"""El Tigre""",NaN,<NA>,NaN,NaN,NaN,NaN,2024-02-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1047,Logan Greenhalgh,"""Tank""",NaN,<NA>,NaN,NaN,NaN,NaN,2024-06-28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
409,Kitt Campbell,NaN,NaN,30,NaN,NaN,NaN,NaN,2025-05-23,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
302,Michael Bonnette,"""No Mercy""",NaN,34,NaN,NaN,NaN,NaN,2025-12-08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
2730,Vineesh Subrahmanyan,"""Vini""",NaN,<NA>,NaN,NaN,NaN,NaN,2025-12-08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
130,Carrese Archer,"""One Punch""",NaN,33,72.0,73.0,NaN,NaN,2025-12-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1716,Justin Martin,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,2025-12-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1771,Sean Mcinerney,"""Poopies""",NaN,36,NaN,NaN,NaN,NaN,2025-12-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1795,Dominik Melendez,"""The Menace""",NaN,<NA>,NaN,NaN,NaN,NaN,2025-12-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True


#### ko_tko/decision/submission wins

In [198]:
# Obtener los nombres de los peleadores cuyas finalizaciones son Nan
ko_nan = fighters[fighters['ko_tko_wins'].isna()]['name']
decisions_nan = fighters[fighters['decision_wins'].isna()]['name']
submission_nan = fighters[fighters['submission_wins'].isna()]['name']

# Verificar que los valores faltantes correspondan a los mismos peleadores en todos los casos
set(ko_nan) == set(decisions_nan) == set(submission_nan)

True

In [199]:
fighters[fighters['ko_tko_wins'].isna()].sort_values(by='ufc_debut_date')

,name,nickname,place_of_birth,age,height,reach,leg_reach,fighting_style,ufc_debut_date,wins,losses,draws,ko_tko_wins,decision_wins,submission_wins,is_active
1257,Enson Inoue,NaN,United States,56,0.0,NaN,NaN,NaN,1997-05-30,0,0,0,<NA>,<NA>,<NA>,True
323,Ebenezer Braga,NaN,"Rio de Janeiro, Brazil",54,0.0,NaN,NaN,NaN,1998-10-16,0,0,0,<NA>,<NA>,<NA>,True
1307,Asikeerbai Jinensibieke,NaN,"Xinjiang, China",34,NaN,NaN,NaN,NaN,2022-06-10,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1705,Dallas Marron,"""El Tigre""",NaN,<NA>,NaN,NaN,NaN,NaN,2024-02-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1047,Logan Greenhalgh,"""Tank""",NaN,<NA>,NaN,NaN,NaN,NaN,2024-06-28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
409,Kitt Campbell,NaN,NaN,30,NaN,NaN,NaN,NaN,2025-05-23,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
302,Michael Bonnette,"""No Mercy""",NaN,34,NaN,NaN,NaN,NaN,2025-12-08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
2730,Vineesh Subrahmanyan,"""Vini""",NaN,<NA>,NaN,NaN,NaN,NaN,2025-12-08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1675,Bilyal Makhov,NaN,"USSR, Russia",35,NaN,NaN,NaN,NaN,2025-12-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True
1689,William Marcario,"""Patolino""",Brazil,31,NaN,NaN,NaN,NaN,2025-12-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True


### Fights

In [215]:
fights.isna().sum()

event                                    0
weight_class                            75
weight                                 199
catch_weight_lbs                      8392
accolades                                0
possible_rounds                          0
card_segment                           616
fight_order                              0
status                                   0
winner                                 149
outcome                               8318
round                                    0
time                                     0
method                                   0
ending_submission                     6811
fight_of_the_night                       0
referee                                  0
red_corner                               0
total_strikes_attempted_red             21
total_strikes_landed_red                21
significant_strikes_attempted_red       21
significant_strikes_landed_red          21
knockdowns_red                          21
head_signif

#### Event

In [201]:
fights = fights.drop(8444).reset_index(drop=True)

#### Weight_class/Catch_weight_lbs

Se justifican los valores NaN ya que fueron peleas en peso pactado

In [202]:
fights[fights['weight_class'].isna()]['catch_weight_lbs'].isna().sum()

0

Se justifican los valores NaN, ya que corresponden a peleas correspondientes a una categoría específica

In [203]:
fights[fights['catch_weight_lbs'].isna()]['weight_class'].isna().sum()

0

#### Weight

Se justifican los valores NaN, ya que fueron peleas de peso libre

In [204]:
fights[(fights['weight'].isna()) & (fights['weight_class'].notna())]['weight_class'].unique()

array(['Open Weight'], dtype=object)

#### Card_segment

No es clara la transición entre eventos sin división en su cartelera y los primeros eventos con preliminares. Hay casos de peleas que cumplian un rol parecido, pero no hay información precisa sobre eso y la base de datos de la UFC no las clasifica en sus archivos JSON, dejandolas como None. Por lo que no asignaré valores solo por suposición.

#### Winner/Outcome

Se justifican los valores NaN, ya que las peleas no pudieron terminar o terminaron en empate

In [205]:
fights[fights['winner'].isna()]['outcome'].unique()

array(['Draw', 'No Contest', nan], dtype=object)

Se justifican los valores NaN, ya que las peleas efectivamente tuvieron un ganador

In [206]:
fights[fights['outcome'].isna()]['winner'].isna().sum()

5

#### Round/Time/Method

In [207]:
fights[fights['round'].isna()][['event', 'red_corner', 'blue_corner', 'status', 'round', 'time', 'method']]

,event,red_corner,blue_corner,status,round,time,method
1049,UFC Fight Night: Dariush vs. Tsarukyan,Dan Hooker,King Green,Canceled,<NA>,NaT,NaN
1383,UFC Fight Night: Song vs. Simon,Brian Kelleher,Journey Newson,Canceled,<NA>,NaT,NaN
2784,UFC Fight Night: Brunson vs. Shahbazyan,Kevin Holland,Trevin Giles,Canceled,<NA>,NaT,NaN
3290,UFC 239: Jones vs. Santos,Francis Ngannou,Junior Dos Santos,Canceled,<NA>,NaT,NaN
3645,UFC 230: Cormier vs. Lewis,Brian Kelleher,Montel Jackson,Canceled,<NA>,NaT,NaN


In [208]:
# Eliminar peleas canceladas
fights = fights.drop(fights[fights['round'].isna()].index).reset_index(drop=True)

#### Ending submission

Se justifican los valores NaN, ya que dichas peleas no terminaron por sumisión y no cuentan con un valor con el que rellenar esa columna

In [209]:
len(fights[fights['method'] != 'Submission']) - len(fights[fights['ending_submission'].isna()])

8

8 filas fueron encontradas donde el valor de la columna 'method' es distinto a 'submission', eso se debe a que dichas peleas terminaro por sumisión, pero luego fueron declaradas No Contest y el método cambiado a 'Overturned', sin eliminar el nombre de la sumisión.

In [210]:
len(fights[(fights['outcome'] == 'No Contest') & (fights['ending_submission'].notna())])

8

#### Red/Blue stats

No se encuentran disponibles las estadísticas de dichas peleas, por lo que sus valores NaN serán dejados tal cual

In [211]:
stats = fights[fights['total_strikes_attempted_red'].isna()]

In [212]:
over = fights[fights['status'] == 'Over']

In [213]:
set(stats) == set(over)

True

In [214]:
red_clinch = fights[fights['clinch_control_time_red'].isna()]
red_ground = fights[fights['ground_control_time_red'].isna()]
blue_clinch = fights[fights['clinch_control_time_blue'].isna()]
blue_ground = fights[fights['ground_control_time_blue'].isna()]

set(red_clinch) == set(blue_clinch) == set(red_ground) == set(blue_ground)

True

# Crear base de datos 

In [ ]:
# Establecer conexión para crear base de datos
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)   

cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS ufc_db")

cursor.close()
conn.close()

## Configurariones

In [ ]:
# Establecer conexión para crear tablas e insertar filas
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="ufc_db"
)
cursor = conn.cursor()

In [26]:
# Diccionario de equivalencias de tipos
mysql_types = {
    "object": "VARCHAR(255)",
    "float64": "FLOAT",
    "int64": "INT",
    "Int64": "INT",
    "bool": "TINYINT(1)",
    "datetime64[ns]": "DATETIME",
    "datetime64[ns, UTC]": "DATETIME",
    "timedelta64[ns]": "TIME"
}

## Crear tablas independientes

### Fighters

In [27]:
fighters_table = {
    'name': 'fighters',
    'id': 'fighter_id',
    'fk': '',
    'pk': 'fighter_id'
}

In [28]:
to_db.create_table(fighters_table, fighters, mysql_types, cursor)
new_fighters = to_db.get_new_rows(fighters_table['name'], fighters, cursor)
to_db.insert_into(fighters_table['name'], new_fighters, cursor, conn)

Tabla vacía, todas las filas podrán ser insertadas


### Submissions

In [29]:
submissions = fights[['ending_submission']].dropna().drop_duplicates()
submissions.rename(columns={'ending_submission': 'name'}, inplace=True)

submissions_table = {
    'name': 'submissions',
    'id': 'submission_id',
    'fk': '',
    'pk': 'submission_id'
}

In [30]:
to_db.create_table(submissions_table, submissions, mysql_types, cursor)
new_submissions = to_db.get_new_rows(submissions_table['name'], submissions, cursor)
to_db.insert_into(submissions_table['name'], new_submissions, cursor, conn)

Tabla vacía, todas las filas podrán ser insertadas


### Weight Classes

In [31]:
weight_classes = fights[fights['weight_class'].notna()][['weight_class', 'weight']].drop_duplicates().sort_values(by='weight')
weight_classes.rename(columns={'weight_class': 'name', 'weight': 'weight_range'}, inplace=True)

weight_classes_table = {
    'name': 'weight_classes',
    'id': 'weight_class_id',
    'fk': '',
    'pk': 'weight_class_id'
}


In [32]:
to_db.create_table(weight_classes_table, weight_classes, mysql_types, cursor)
to_db.insert_into(weight_classes_table['name'], weight_classes, cursor, conn)

### Referees

In [33]:
referees = fights[['referee']]
referees.rename(columns={'referee': 'name'}, inplace=True)

referees_table = {
    'name': 'referees',
    'id': 'referee_id',
    'fk': '',
    'pk': 'referee_id'
}

C:\Users\facun\AppData\Local\Temp\ipykernel_16916\1014917402.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  referees.rename(columns={'referee': 'name'}, inplace=True)


In [34]:
to_db.create_table(referees_table, referees, mysql_types, cursor)
new_referees = to_db.get_new_rows(referees_table['name'], referees, cursor)
to_db.insert_into(referees_table['name'], new_referees, cursor, conn)

Tabla vacía, todas las filas podrán ser insertadas


### Venues

In [35]:
venues = events[['venue', 'country', 'state', 'city']].drop_duplicates()
venues.rename(columns={'venue': 'name'}, inplace=True)

venues_table = {
    'name': 'venues',
    'id': 'venue_id',
    'fk': '',
    'pk': 'venue_id'
}

In [36]:
to_db.create_table(venues_table, venues, mysql_types, cursor)
new_venues = to_db.get_new_rows(venues_table['name'], venues, cursor)
to_db.insert_into(venues_table['name'], new_venues, cursor, conn)

Tabla vacía, todas las filas podrán ser insertadas


## Crear tablas dependientes

### Events

#### Cambias nombres de columnas en events y reemplazar sus valores por IDs

In [37]:
# Renombrar llaves foráneas
events.rename(columns={'venue': 'venue_id'}, inplace=True)

In [38]:
# Reemplazar nombre del venue por su ID
events['venue_id'] = events['venue_id'].apply(lambda x: to_db.get_id('venue_id', 'venues', 'name', x, cursor))

#### Crear tabla

In [39]:
to_events = events[['name', 'date', 'venue_id']]
events_table = {
    'name': 'events',
    'id': 'event_id',
    'fk': [('venue_id', 'venues', 'venue_id')],
    'pk': 'event_id'
}

In [40]:
to_db.create_table(events_table, to_events, mysql_types, cursor)
new_events = to_db.get_new_rows(events_table['name'], to_events, cursor)
to_db.insert_into(events_table['name'], new_events, cursor, conn)

Tabla vacía, todas las filas podrán ser insertadas


### Fights

#### Cambiar nombres de columnas en fights y reemplazar sus valores por IDs

In [41]:
# Renombrar llaves foráneas
fights.rename(columns={
    'event': 'event_id', 
    'weight_class':'weight_class_id',
    'winner': 'winner_id',
    'ending_submission': 'submission_id',
    'referee': 'referee_id',
    'red_corner': 'red_corner_id', 
    'blue_corner': 'blue_corner_id'
    }, inplace=True)

In [42]:
# Reemplazar nombre del evento por su ID
fights['event_id'] = fights['event_id'].apply(lambda x: to_db.get_id('event_id', 'events', 'name', x, cursor))

In [43]:
# Reemplazar el nombre de la categoría por su ID
fights['weight_class_id'] = fights['weight_class_id'].apply(lambda x: to_db.get_id('weight_class_id',  'weight_classes', 'name', x, cursor))

In [44]:
# Reemplazar el nombre del ganador por su ID
fights['winner_id'] = fights['winner_id'].apply(lambda x: to_db.get_id('fighter_id', 'fighters', 'name', x, cursor))

In [45]:
# Reemplazar el nombre de la sumisión por su ID
fights['submission_id'] = fights['submission_id'].apply(lambda x: to_db.get_id('submission_id', 'submissions', 'name', x, cursor))

In [46]:
# Reemplazar el nombre del referee por su ID
fights['referee_id'] = fights['referee_id'].apply(lambda x: to_db.get_id('referee_id', 'referees', 'name', x, cursor))

In [47]:
# Reemplazar esquina roja por su ID
fights['red_corner_id'] = fights['red_corner_id'].apply(lambda x: to_db.get_id('fighter_id', 'fighters', 'name', x, cursor))

In [48]:
# Reemplazar esquina azul por su ID
fights['blue_corner_id'] = fights['blue_corner_id'].apply(lambda x: to_db.get_id('fighter_id', 'fighters', 'name', x, cursor))

In [49]:
# Cambiar tipo de dato a Int64
cols = ['event_id', 'weight_class_id', 'winner_id', 'submission_id', 'referee_id', 'red_corner_id', 'blue_corner_id']
fights[cols] = fights[cols].astype('Int64')

#### Crear tabla

In [50]:
to_fights = fights.drop(columns=['weight'])
fights_table = {
    'name': 'fights',
    'id': 'fight_id',
    'fk': [('event_id', 'events', 'event_id'),
           ('weight_class_id', 'weight_classes', 'weight_class_id'),
           ('winner_id', 'fighters', 'fighter_id'),
           ('submission_id', 'submissions', 'submission_id'),
           ('referee_id', 'referees', 'referee_id'),
           ('red_corner_id', 'fighters', 'fighter_id'),
           ('blue_corner_id', 'fighters', 'fighter_id')],
    'pk': 'fight_id'
}

In [51]:
to_db.create_table(fights_table, to_fights, mysql_types, cursor)
new_fights = to_db.get_new_rows(fights_table['name'], to_fights, cursor)
to_db.insert_into(fights_table['name'], new_fights, cursor, conn)

Tabla vacía, todas las filas podrán ser insertadas
